In [93]:
import numpy as np
import pandas as pd
import os
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [94]:
# Load data
file_path = 'Turbofan_HPC_Efficiency.csv'
hpc_data = pd.read_csv(file_path)

In [ ]:
# Feature selection
correlation = hpc_data.corr()
print(correlation['NetThrust_kN'])

In [ ]:
columns_to_drop = ['LPTExitPressureP5_kPA', 'Isentr.HPCEfficiency', 'LPTExitTemperatureT5_K', 
                   'LPSpoolSpeed_RPM', 'HPSpoolSpeed_RPM', 'EnginePressureRatioP5/P2', 
                   'FuelFlow_kg/s', 'CoreNozzlePressureRatio', 'BypassNozzlePressureRatio']
columns_to_drop = [col for col in columns_to_drop if col in hpc_data.columns]
hpc_data = hpc_data.drop(columns=columns_to_drop)

# Define features and target
X = hpc_data.drop(columns=['NetThrust_kN'])
y = hpc_data['NetThrust_kN']

# Display the first few rows of the dataset
print(hpc_data.head())


In [ ]:
# EDA
sns.pairplot(hpc_data, x_vars=['CoreNozzleGrossThrust_kN', 'BypassNozzleGrossThrust_kN', 
                               'Sp.FuelConsumption_g/(kN*s)', 'SpecificThrust_m/s', 
                               'CoreNozzleVel.V8_m/s', 'BypassNozzleVel.V18_m/s', 
                               'BurnerEfficiency'], y_vars='NetThrust_kN', height=7, aspect=0.7, kind='reg')
plt.show()

In [98]:
# Split data
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=50)

In [ ]:
# Define the model
model = RandomForestRegressor(random_state=50)

# Define the parameters for exploration
param_grid = {'n_estimators': [50, 100, 250]}

# Perform Grid Search with parallel processing and verbose output
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
grid_search.fit(train_X, train_y)

# Get the optimal number of estimators
best_n = grid_search.best_params_['n_estimators']

# Fit the model with the optimal number of estimators
final_model = RandomForestRegressor(n_estimators=best_n, random_state=50, n_jobs=-1)
final_model.fit(train_X, train_y)

In [ ]:
# Make validation predictions and calculate mean absolute error
val_predictions = final_model.predict(test_X)
val_mae = mean_absolute_error(test_y, val_predictions)
val_mse = mean_squared_error(test_y, val_predictions)
val_r2 = r2_score(test_y, val_predictions)

# Print the metrics
print("Validation MAE for best tree size: {:,.10f}".format(val_mae))
print("Validation MSE for best tree size: {:,.10f}".format(val_mse))
print("Validation R^2 for best tree size: {:,.10f}".format(val_r2))

# Cross validation
scores = cross_val_score(final_model, train_X, train_y, cv=3, n_jobs=-1)
print("Cross validation score: ", np.mean(scores))

In [ ]:
# Feature importance
importances = final_model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print(feature_importance_df)

### Scenario Analysis

In [ ]:
scenario_data = test_X.copy()

# Example adjustment: Increase top 3 features by 5%
adjustment_factor = 1.05
top_features = feature_importance_df['Feature'][:3].tolist()  # Adjust top 3 features and convert to list

# Ensure correct slicing
if top_features:
    for feature in top_features:
        scenario_data[feature] = scenario_data[feature] * adjustment_factor

# Predict the net thrust for the new scenarios
scenario_pred = final_model.predict(scenario_data)

# Calculate the percentage improvement
improvement = (scenario_pred - val_predictions) / val_predictions * 100
average_improvement = np.mean(improvement)
print(f"Average Improvement in Net Thrust: {average_improvement:.2f}%")

# Plot the original vs. scenario predictions
plt.figure(figsize=(10, 6))
plt.plot(val_predictions, label='Original Predictions', alpha=0.7)
plt.plot(scenario_pred, label='Scenario Predictions', alpha=0.7)
plt.xlabel('Samples')
plt.ylabel('Net Thrust (kN)')
plt.legend()
plt.title('Original vs Scenario Predictions')
plt.show()

# Analyze impact on other features (optional)
# Calculate the percentage change in features
feature_changes = (scenario_data - test_X) / test_X * 100
average_feature_changes = feature_changes.mean()
print("Average Feature Changes:\n", average_feature_changes)